In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from google.colab.patches import cv2_imshow # colab内で画像表示関数がうまく動かないので、パッチが提供されている

# Googleドライブへのマウント
from google.colab import drive
drive.mount('/content/drive')
%cd "/content/drive/My Drive/Colab Notebooks/ou_dip/"

In [ ]:
# 自分でカーネルを作る例

img = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み（グレースケール）

kernel = np.ones((5,5),np.float32)/25 # 5x5平均化フィルタ
dst = cv2.filter2D(img,-1,kernel) # ddepth=-1とした場合、入出力のchannel数はおなじになる

cv2_imshow(dst)

In [ ]:
# OpenCV組み込み関数を使う例

img = cv2.imread("sample.jpg", cv2.IMREAD_GRAYSCALE) # 画像読み込み（グレースケール）

dx = cv2.Sobel(img,cv2.CV_32F,1,0) # x方向の微分
dy = cv2.Sobel(img,cv2.CV_32F,0,1) # y方向の微分

mag = cv2.magnitude(dx, dy)  # 勾配強度の計算
ori = cv2.phase(dx, dy, angleInDegrees=True)  # 勾配方向の計算

# HSVで可視化 参考：https://stackoverflow.com/questions/51667520/gradient-orientation-in-opencv
vis_hsv = np.zeros((img.shape[0],img.shape[1],3),dtype=np.uint8)
vis_hsv[..., 0] = ori / 2 # H (0～180に正規化)
vis_hsv[..., 1] = 255 # S
vis_hsv[..., 2] = cv2.normalize(mag, None, 0, 255, cv2.NORM_MINMAX) # V
vis_bgr = cv2.cvtColor(vis_hsv, cv2.COLOR_HSV2BGR)

cv2_imshow(vis_bgr)